In [ ]:
import argparse
import multiprocessing
import os
from itertools import product

import numpy as np
import yaml

from config import SPECS_COMPLETE_YAMLS, WS_RESULTS_DIR
from model import DC, LPAC, Solution
from util import Grid, ResilienceState, in_notebook, max_budget_by_omega

In [ ]:
# user inputs
if in_notebook():
    f_min = 0
    f_max = 20
    f_inc = 1
    r_hat = 3
    pftype = 'dc'
    casestudy = 'imelda'
    num_processes = 8
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--fmin')
    parser.add_argument('--fmax')
    parser.add_argument('--finc')
    parser.add_argument('--rhat')
    parser.add_argument('--casestudy')
    parser.add_argument('--pftype')
    parser.add_argument('--numprocesses')
    args = parser.parse_args()
    f_min = int(args.fmin)
    f_max = int(args.fmax)
    f_inc = int(args.finc)
    r_hat = int(args.rhat)
    pftype = str(args.pftype)
    casestudy = str(args.casestudy)
    num_processes = int(args.numprocesses)

In [ ]:
hw_threads = multiprocessing.cpu_count()
gurobi_threads = int(np.floor(hw_threads / num_processes))

In [ ]:
DEGREE = 1
WEIGHTS_LOAD = [1.000]
WEIGHTS_FLOW = [0.000, 0.025, 0.050, 0.075, 0.100, 0.125, 0.150]
OVERSPEND = False

In [ ]:
model_classes = {
    'dc': DC,
    'lpacc': LPAC,
    'lpacf': LPAC,
    'qpac': LPAC
}

In [ ]:
# load the data
with open(SPECS_COMPLETE_YAMLS[casestudy, pftype]) as fh:
    specs = yaml.load(fh, Loader=yaml.Loader)
    for key, val in specs['r_hat'].items():
        specs['r_hat'][key] = min(val, r_hat)
    for key in list(specs['xi']):
        (k, r, omega) = key
        if r > r_hat:
            specs['xi'].pop(key)
    for k in specs['R']:
        specs['R'][k] = [i for i in range(1, min(max(specs['R'][k]), r_hat) + 1)]
specs['options']['approach'] = 'stochastic'  # arbitrary since there is only one scenario

In [ ]:
def solve_wait_and_see(modelcls, omega, specs):
    specs_copy = specs.copy()
    f_req = max_budget_by_omega(**specs_copy)[omega]
    probability = specs_copy.pop('probability')
    Omega = specs_copy.pop('Omega')
    xi = specs_copy.pop('xi')
    specs_copy['Omega'] = {omega}
    specs_copy['probability'] = {omega: 1.0}
    specs_copy['xi'] = {(k, r, omega_prime): xi[k, r, omega]
                        for (k, r, omega_prime) in xi
                        if omega_prime == omega}
    MODEL = modelcls(**specs_copy)
    MODEL.model.setParam('MIPGap', 0.0)
    MODEL.model.setParam('PreSolve', 2)
    MODEL.model.setParam('OutputFlag', 0)
    MODEL.model.setParam('Threads', gurobi_threads)
    MODEL.model.setParam('MIPFocus', 2)
    MODEL.model.setParam('ImpliedCuts', 2)
    MODEL.model.setParam('ProjImpliedCuts', 2)

    grid = Grid(specs_copy)
    rstate = ResilienceState(grid)

    for f in np.arange(f_min, min(f_max, f_req) + f_inc, f_inc):

        MODEL.con_resource_hi.RHS = f
        MODEL.update()

        MODEL.model.NumStart = len(WEIGHTS_LOAD) * len(WEIGHTS_FLOW) + (1 if f > 0 else 0)
        for i, (weight_load, weight_flow) in enumerate(product(WEIGHTS_LOAD, WEIGHTS_FLOW)):
            rstate.reset()
            sol = rstate.adopt_stochastic_greedy_solution(f, DEGREE, weight_load, weight_flow, OVERSPEND)
            MODEL.model.setParam('StartNumber', i)
            for (k, r) in MODEL.x:
                MODEL.x[(k, r)].Start = 1 if (k, r) in sol else 0
        if f > 0:
            MODEL.model.setParam('StartNumber', i + 1)
            last_zipfile = os.path.join(WS_RESULTS_DIR, f'{casestudy}-{pftype}-f{f-1}-omega{omega}-r{r_hat}.zip')
            solution = Solution.from_zip(last_zipfile)
            for key, val in solution['x'].items():
                MODEL.x[key].Start = val
        MODEL.update()

        os.makedirs(WS_RESULTS_DIR, exist_ok=True)
        zipfile = os.path.join(WS_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-omega{omega}-r{r_hat}.zip')
        if not os.path.exists(zipfile):
            print(omega, f)
            MODEL.solve()
            solution = Solution.from_solved_instance(MODEL)
            solution.to_zip(zipfile, variables=['x', 'ObjVal', 'ObjBound', 'gamma', 'gamma_under', 'gamma_over'])

In [ ]:
args = [(model_classes[pftype], omega, specs)
        for omega in specs['Omega'].copy()]

with multiprocessing.Pool(num_processes) as pool:
    pool.starmap(solve_wait_and_see, args)